In [160]:
# CUDA_LAUNCH_BLOCKING=1
# TORCH_USE_CUDA_DSA

In [161]:
data_dir = '/kaggle/input/turing-patterns/Monidipa_Shubham'
spiral_dir = '/kaggle/input/spirals'
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(3),  # Convert to RGB (3 channels)
    transforms.ToTensor(),
#     transforms.Normalize(mean=[0.5], std=[0.5]),
])

dataset = datasets.ImageFolder(root = data_dir, transform=transform)
class_names = dataset.classes
print(class_names)

['Spirals', 'Spots', 'Strips']


In [162]:
loader = DataLoader(dataset, batch_size=64, shuffle=True)

mean_sum = torch.zeros(3)
std_sum = torch.zeros(3)
count = 0

# Calculate mean and standard deviation for each channel
for images, _ in loader:
    batch_size = images.size(0)
    count += batch_size
    mean_sum += images.mean(dim=[0, 2, 3]) * batch_size
    std_sum += images.std(dim=[0, 2, 3]) * batch_size

# Calculate the overall mean and standard deviation
mean = mean_sum / count
std = std_sum / count

print(f"Estimated Mean: {mean}")
print(f"Estimated Std: {std}")
    




Estimated Mean: tensor([0.7122, 0.7122, 0.7122])
Estimated Std: tensor([0.2701, 0.2701, 0.2701])


In [163]:
normalize_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(3),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

dataset = datasets.ImageFolder(root=data_dir, transform=normalize_transform)
spirals = datasets.ImageFolder(root=spiral_dir, transform=normalize_transform)
# print(data.class_to_idx)
# for image, labels in dataloader:
#     print(labels)

In [164]:
train_size = int(len(dataset)*.7)
# valid_size = int(len(dataset)*.2)
test_size = int(len(dataset) - train_size)

train, test = random_split(dataset, [train_size, test_size])

batch_size = 32
train_dataloader = DataLoader(train, batch_size=batch_size, shuffle=True)
# valid_dataloader = DataLoader(valid, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test, batch_size=batch_size, shuffle=False)
# spiral_dataloader = DataLoader(spirals, batch_size=batch_size, shuffle=False)
# transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.Grayscale(3),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.5], std=[0.5]),
# ])

# mnist_train_dataset = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
# mnist_test_dataset = datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)

# train_loader = DataLoader(mnist_train_dataset, batch_size=32, shuffle=True)
# test_loader = DataLoader(mnist_test_dataset, batch_size=32, shuffle=False)

In [165]:
# class CNNModel(nn.Module):
#     def __init__(self, num_classes = 3):
#         super(CNNModel, self).__init__()
        
#         self.conv1 = nn.Conv2d(3, 64, kernel_size = 3, stride = 1)
#         self.bn1 = nn.BatchNorm2d(64)
#         self.conv2 = nn.Conv2d(64, 128, kernel_size = 3, stride =1 )
#         self.bn2 = nn.BatchNorm2d(128)
#         self.conv3 = nn.Conv2d(128, 256, kernel_size = 3, stride = 1)
#         self.bn3 = nn.BatchNorm2d(256)
        
#         self.relu = nn.ReLU()
#         self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
#         self.fc1 = nn.Linear(256*26*26, 512)
#         self.bn4 = nn.BatchNorm1d(512)
#         self.fc2 = nn.Linear(512, num_classes)
#         self.bn5 = nn.BatchNorm1d(num_classes)
        
#         self.softmax = nn.Softmax(dim=1)
    
#     def forward(self, x):
# #         print(x.size())
#         x = self.pool(self.relu(self.bn1(self.conv1(x))))
# #         print(x.size())
#         x = self.pool(self.relu(self.bn2(self.conv2(x))))
# #         print(x.size())
#         x = self.pool(self.relu(self.bn3(self.conv3(x))))
# #         print(x.size())
#         x = x.view(-1, 256*26*26)
#         print(x.size())
#         x = self.relu(self.bn4(self.fc1(x)))
#         print(x.size())
#         x = self.bn5(self.fc2(x))
#         x = F.log_softmax(x, 1)
# #         print(x.size())
        
#         return x

# model = CNNModel().to('cuda')
# criterion = nn.CrossEntropyLoss()
# optimiser = optim.Adam(model.parameters(), lr=0.001)

alexnet_model = models.vgg16(pretrained=True)

num_features = alexnet_model.classifier[6].in_features
alexnet_model.classifier[6] = nn.Linear(num_features, 3)
alexnet_model = alexnet_model.to("cuda")
alexnet_model.train()

# num_ftrs = alexnet_model.fc.in_features
# alexnet_model.fc = torch.nn.Linear(num_ftrs, 3)
# alexnet_model = alexnet_model.to("cuda")
# alexnet_model.train()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [166]:
criterion = nn.CrossEntropyLoss()
optimiser = optim.RMSprop(alexnet_model.parameters(), lr=0.001)

In [167]:
num_epoch = 5
# for epoch in range(num_epoch):
#     alexnet_model.train()
#     train_loss = 0.0
#     for images, labels in train_dataloader:
#         images, labels = images.to('cuda'), labels.to('cuda')
#         optimiser.zero_grad()
#         output = alexnet_model(images)
# #         output = output.float()
# #         labels = labels.long()
# #         print(output)
#         loss = criterion(output, labels)
#         loss.backward()
#         optimiser.step()
#         total_loss = loss.item()
        
#     avg_loss = total_loss/len(train_dataloader)
#     print(f'Training Epoch [{epoch+1}/{num_epoch}], Loss: {avg_loss}')
    


for epoch in range(num_epoch):
    for inputs, labels in train_dataloader:
        optimiser.zero_grad()
        inputs, labels = inputs.to("cuda"), labels.to("cuda")
        outputs = alexnet_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimiser.step()
        
    print(f"Epoch [{epoch+1}/{num_epoch}], Loss: {loss.item()}")
    
# torch.save(model.state_dict(), 'custom_cnn_model_gpu.pth')

Epoch [1/5], Loss: 10035682304.0
Epoch [2/5], Loss: 6623.23095703125
Epoch [3/5], Loss: 3207.948486328125
Epoch [4/5], Loss: 98.7304458618164
Epoch [5/5], Loss: 226.54747009277344


For AlexNet
Epoch [1/5], Loss: 0.0
Epoch [2/5], Loss: 0.0
Epoch [3/5], Loss: 0.0
Epoch [4/5], Loss: 0.0
Epoch [5/5], Loss: 0.0

For ResNet
Epoch [1/5], Loss: 2.1291191577911377
Epoch [2/5], Loss: 2.1237082481384277
Epoch [3/5], Loss: 2.1333234310150146
Epoch [4/5], Loss: 2.130035161972046
Epoch [5/5], Loss: 2.127274751663208

In [168]:
alexnet_model.eval()
for epoch in range(num_epoch):
    
    correct = 0
    total = 0
    for images, labels in test_dataloader:
#         print(images)
#         fig, axes = plt.subplots(1,1)
#         axes.imshow(images)
#         plt.show()
#         plt.imshow(images)
#         t_i = normalize_transform(images)
#         actual_batch_size = images.size(0)
#         plt.figure(figsize=(15, 15))  # Adjust figure size as needed
#         for i in range(actual_batch_size):
#             plt.subplot(4, 8, i + 1)  # Assuming 32 images will be displayed in a 4x8 grid
# #             plt.imshow(transforms.ToPILImage()(images[i]))
#             plt.imshow((images[i].permute(1, 2, 0)))# Convert tensor back to PIL Image for plotting
#             plt.title(f'Label: {class_names[labels[i]]}')
#             plt.axis('off')
#         plt.show()
        images, labels = images.to('cuda'), labels.to('cuda')
        
        optimiser.zero_grad()
        output = alexnet_model(images)
        i, predicted = torch.max(output.data, 1)
        
#         print(labels)
#         for i in range(actual_batch_size):
#             print(class_names[predicted[i]])
#         print(predicted)
        correct += predicted.eq(labels).sum().item()
        total += labels.size(0)
    
    accuracy = correct/total
    print(f"Epoch {epoch + 1}/{num_epoch}, Validation Accuracy: {accuracy * 100:.2f}%")

Epoch 1/5, Validation Accuracy: 66.67%
Epoch 2/5, Validation Accuracy: 66.67%
Epoch 3/5, Validation Accuracy: 66.67%
Epoch 4/5, Validation Accuracy: 66.67%
Epoch 5/5, Validation Accuracy: 66.67%


For AlexNet
Epoch 1/5, Validation Accuracy: 100.00%
Epoch 2/5, Validation Accuracy: 100.00%
Epoch 3/5, Validation Accuracy: 100.00%
Epoch 4/5, Validation Accuracy: 100.00%
Epoch 5/5, Validation Accuracy: 100.00%

For ResNet
Epoch 1/5, Validation Accuracy: 0.00%
Epoch 2/5, Validation Accuracy: 0.00%
Epoch 3/5, Validation Accuracy: 0.00%
Epoch 4/5, Validation Accuracy: 0.00%
Epoch 5/5, Validation Accuracy: 0.00%

In [169]:
# alexnet_model.eval()
# for epoch in range(num_epoch):
    
#     correct = 0
#     total = 0
#     for images, labels in spiral_dataloader:
# #         print(images)
# #         fig, axes = plt.subplots(1,1)
# #         axes.imshow(images)
# #         plt.show()
# #         plt.imshow(images)
# #         t_i = normalize_transform(images)
#         actual_batch_size = images.size(0)
#         plt.figure(figsize=(15, 15))  # Adjust figure size as needed
#         for i in range(actual_batch_size):
#             plt.subplot(4, 8, i + 1)  # Assuming 32 images will be displayed in a 4x8 grid
# #             plt.imshow(transforms.ToPILImage()(images[i]))
#             plt.imshow((images[i].permute(1, 2, 0)))# Convert tensor back to PIL Image for plotting
#             plt.title(f'Label: {class_names[labels[i]]}')
#             plt.axis('off')
#         plt.show()
#         images, labels = images.to('cuda'), labels.to('cuda')
        
#         optimiser.zero_grad()
#         output = alexnet_model(images)
#         i, predicted = torch.max(output.data, 1)
        
# #         print(labels)
#         for i in range(actual_batch_size):
#             print(class_names[predicted[i]])
# #         print(predicted)
#         correct += predicted.eq(labels).sum().item()
#         total += labels.size(0)
    
#     accuracy = correct/total
#     print(f"Epoch {epoch + 1}/{num_epoch}, Validation Accuracy: {accuracy * 100:.2f}%")

In [170]:
# alexnet_model.eval()
# y_true=[]
# y_pred=[]
# with torch.no_grad():
#     for test_data in test_dataloader:
#         images, labels = images.to('cuda'), labels.to('cuda')
#         pred = alexnet_model(images).argmax(dim=1)
#         for i in range(len(pred)):
#             y_true.append(labels[i].item())
#             y_pred.append(pred[i].item())
# print(len(y_true))
# print(classification_report(y_true,y_pred,target_names=class_names,digits=4))